TIF images → `Extract-Channels` → `Visualize-Inputs` → `Predict-Segments` → `Visualize-Segmentation` → predicted segmentations

# Setup

In [1]:
import os
import timeit

In [2]:
# Parameters (overridden by notebook parameters)

root_path = '../sample-data/deepcell/mesmer-sample-3'
model_path = os.path.expanduser('~') + '/.keras/models/MultiplexSegmentation'

In [3]:
input_channels_path = '{}/input_channels.npz'.format(root_path)
output_path = root_path

### Load Mesmer model & create app

In [4]:
from imaging_helpers import make_app

# Assumes that the model has been downloaded already.
app = make_app(model_path=model_path)

2023-11-03 18:27:46.647618: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded model in 24.941387450002367 s


### Read input files

In [5]:
import numpy as np

# An array of shape [height, width, 2] containing nuclear & membrane channels
with np.load(input_channels_path) as loader:
    input_channels = loader['input_channels']

# Conform to DeepCell's expected shape [num_inputs, height, width, 2]
input_channels = input_channels[np.newaxis, ...]

# Generate predictions

This step is expensive!

In [6]:
t = timeit.default_timer()

import logging
logging.root.setLevel(logging.DEBUG)

predictions = app.predict(input_channels, image_mpp=0.5)[0]

print('Prediction finished in %s s' % (timeit.default_timer() - t))

Prediction finished in 4.256865661998745 s


# Save predictions

In [7]:
import numpy as np
np.savez_compressed('{}/predictions.npz'.format(output_path), predictions=predictions)